In [ ]:
import tensorflow as tf
import numpy as np
from PIL import ImageGrab
import matplotlib.pyplot as plt
import cv2
import time
import ctypes
import keyboard
import win32gui
from IPython.display import clear_output

#Our files
tf.reset_default_graph()
from keypress import *
from tools import *
#import agent002
import movementagent03


log_addr = r'C:\Users\Ozan\AppData\Roaming\.minecraft\logs\latest.log'
window_title = 'Minecraft 1.7.2'

array_size =  (200, 200, 2)
tf.reset_default_graph()
hwnd = win32gui.FindWindowEx(None, None, None, window_title)
win32gui.MoveWindow(hwnd,-8,0,600,400, 1)

#Initialize agent
agent = movementagent03.Agent(array_size = array_size, lr = 0.01)
#agent.init_model(lr = 0.001)

losses = []
walk_backward = False
walk_count = 0

# Allocate space for data
data_size = 60
data_X = np.zeros((data_size, array_size[0], array_size[1], array_size[2]))
data_y = np.zeros((data_size, 2))

training = True

In [ ]:
#Test Accuracy
epoch_count = 0
def get_test_acc(agent, X_test, y_test):
    preds = agent.predict(X_test)
    test_preds = preds[0]

    test_preds_int = np.argmax(test_preds, axis = 1)
    accuracy = np.mean(test_preds_int == np.argmax(y_test, axis = 1))
    return accuracy

def train_batch(agent, data_X, data_y):
    global data_size
    global epoch_count
    train_size = 40
    # Shuffle data
    shuffle_idx = np.random.permutation(data_size)
    data_X = data_X[shuffle_idx]
    data_y = data_y[shuffle_idx]

    X_train = data_X[:train_size]
    y_train = data_y[:train_size]
    X_test = data_X[train_size:]
    y_test = data_y[train_size:]
    
    loss_train = agent.train_model(X_train, y_train)
    acc_test = get_test_acc(agent, X_test, y_test)
    clear_output()
    print("Epoch {}, train_loss: {}, test_acc: {}".format(epoch_count, loss_train, acc_test))
    epoch_count += 1
    
def acc_batch(agent, data_X, data_y):
    global data_size
    global epoch_count
    
    acc_test = get_test_acc(agent, data_X, data_y)
    clear_output()
    print("Epoch {}, test_acc: {}".format(epoch_count, acc_test))
    epoch_count += 1
    


In [ ]:
out = None

def main(run_time = 5):
    n_loop = 0
    
    walk_backward = False
    walk_count = 0
    pressed_key = None
    
    x,y,z = [None, None, None]
    #print('window box: {}'.format(window_box))
    for i in list(range(4))[::-1]:
        print(i+1)
        time.sleep(1)
    last_time = time.time()
    start_time = last_time
    
    prev_screen = np.zeros((1, array_size[0], array_size[1], 1))
    
    while True:
        win_box, win_box_orig = get_window_coord(window_title)
        new_screen =  np.flip(np.array(ImageGrab.grab(bbox=win_box)), axis = 2)
        
        #Resize new screen to array size
        new_screen = cv2.resize(new_screen,(array_size[0], array_size[1]))
        
        #Convert to grayscale
        new_screen = cv2.cvtColor(new_screen.squeeze(), cv2.COLOR_BGR2GRAY).reshape(1, array_size[0], array_size[1], 1)
        agent_input = np.concatenate([new_screen, prev_screen], axis = 3)
        
        
        movement = np.zeros((1,2))
        """if pressed_key == 'W':
            movement[0, 0] = 1
        elif pressed_key == 'S':
            movement[0, 1] = 1"""
        
        add_data = False
        if keyboard.is_pressed('W'):
            movement[0, 0] = 1
            add_data = True
        elif keyboard.is_pressed('S'):
            movement[0, 1] = 1
            add_data = True
        # Store data
        if add_data:
            if n_loop < data_size:
                data_X[n_loop] = agent_input[0] / 255.0
                data_y[n_loop] = movement[0]
            else:
                #print('Data array is full, train for one batch')
                if training:
                    train_batch(agent, data_X, data_y)
                else:
                    acc_batch(agent, data_X, data_y)
                n_loop = 0
                
        
        #loss = agent.step(agent_input, movement)
        prev_screen = new_screen.copy()
        #losses.append(loss)
        
        # Walking coordination #
        """key_to_press = None
        if walk_backward:
            key_to_press = 'S'
            walk_count += 1
            if walk_count == 100:
                walk_backward = not walk_backward
                walk_count = 0
        else:
            key_to_press = 'W'
            walk_count += 1
            if walk_count == 100:
                walk_backward = not walk_backward
                walk_count = 0
        ############################
        
        if key_to_press != pressed_key:
            if pressed_key is not None:
                keyboard.release(pressed_key)
            else:
                keyboard.press(key_to_press)
                keyboard.release(key_to_press)
                
            keyboard.press(key_to_press)
            pressed_key = key_to_press
            print(key_to_press)"""
        
        MoveMouse(int((win_box_orig[0] + win_box_orig[2]) / 2), int((win_box_orig[1]+ win_box_orig[3]) / 2))
        #keyboard.press_and_release('space')
        #ReleaseKey(W)
        n_loop+=1
        if (n_loop %20) == 0:
            pass
            #x,y,z = read_coord(log_addr)
            #print('position: {}, {}, {}'.format(x,y,z))
        #PressKey(W)
        
        last_time = time.time()
        #cv2.imshow('window_mc', output) #new_screen
        #hwnd = win32gui.FindWindowEx(None, None, None, 'window_mc')
        #win32gui.MoveWindow(hwnd,582,0,array_size[0],array_size[1], 1)
        #cv2.imshow('window',cv2.cvtColor(screen, cv2.COLOR_BGR2RGB))
        if (keyboard.is_pressed('Q')):
            break
        
        if (cv2.waitKey(25) & 0xFF == ord('q')) | (last_time - start_time > run_time):
            cv2.destroyAllWindows()
            break

In [ ]:
main(2000)
"""clear_output()
plt.figure(figsize = (8, 16))
plt.plot(losses[10:])"""

In [ ]:
training= False

In [ ]:
i = 9
np.save('movement_data_X_{}.npy'.format(i), data_X)
np.save('movement_data_y_{}.npy'.format(i), data_y)